In [1]:
import os
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [2]:
# Set seed for reproductability
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

In [3]:
# Step 1: Load the historical data
data = pd.read_csv('historical_data.csv')


In [4]:
# Step 2: Load the sentiment scores data
sentiment_data = pd.read_csv('daily_sentiment_scores_both.csv')

In [5]:
# Step 3: Filter the required columns and sort by date and ticker
data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Ticker']]
data['Date'] = pd.to_datetime(data['Date'])
data.sort_values(['Ticker', 'Date'], inplace=True)

In [6]:
# Step 4: Merge sentiment scores (neutral, positive, negative) with historical data
sentiment_data['datetime'] = pd.to_datetime(sentiment_data['datetime'])
data = pd.merge(data, sentiment_data[['datetime', 'neutral', 'positive', 'negative']], left_on='Date', right_on='datetime', how='left')

# Drop extra datetime column
data.drop(columns=['datetime'], inplace=True)

# Fill NaN sentiment scores with 0 (assume neutral sentiment if missing)
data[['neutral', 'positive', 'negative']] = data[['neutral', 'positive', 'negative']].fillna(0)

In [7]:
# Step 5: Encode the Ticker column
encoder = OneHotEncoder(sparse=False)
tickers_encoded = encoder.fit_transform(data[['Ticker']])

c:\Users\astev\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [8]:
# Step 6: Add the encoded tickers as additional features
encoded_columns = [f'Ticker_{i}' for i in range(tickers_encoded.shape[1])]
encoded_df = pd.DataFrame(tickers_encoded, columns=encoded_columns, index=data.index)
data = pd.concat([data.reset_index(drop=True), encoded_df], axis=1)

In [9]:
# Step 7: Scale numerical features
scaler_features = MinMaxScaler(feature_range=(0, 1))
numerical_features = ['Open', 'High', 'Low', 'Volume']  # Exclude 'Close' from general scaling
data[numerical_features] = scaler_features.fit_transform(data[numerical_features])

# Separate scaler for 'Close' price
scaler_close = MinMaxScaler(feature_range=(0, 1))
data[['Close']] = scaler_close.fit_transform(data[['Close']])  # Scale only 'Close' separately

In [10]:
# The sentiment columns are already between 0 and 1, so we don't scale them again
numerical_features_with_sentiment = numerical_features + ['neutral', 'positive', 'negative']

In [11]:
# Step 8: Preprocessing function
def preprocess_data(data, n_steps=10):
    X, y = [], []
    feature_columns = numerical_features_with_sentiment + ['Close'] + encoded_columns

    for ticker in data['Ticker'].unique():
        ticker_data = data[data['Ticker'] == ticker]  # Ensure sequences stay within the same stock
        ticker_data = ticker_data[feature_columns].values

        for i in range(n_steps, len(ticker_data)):
            X.append(ticker_data[i - n_steps:i, :])  # Last n_steps rows as features
            y.append(ticker_data[i, -len(encoded_columns)-1])  # Predict 'Close' price

    return np.array(X), np.array(y)

In [12]:
# Step 9: Preprocess the data
X, y = preprocess_data(data, n_steps=10)

In [13]:
# Step 10: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Step 11: Build the LSTM model
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25, activation='relu'),
    Dense(1)  
])

c:\Users\astev\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
# Step 12: Compile the model
model.compile(optimizer='adam', loss='mse')

In [16]:
# Step 13: Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1)


Epoch 1/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.0012 - val_loss: 1.2071e-04
Epoch 2/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 2.3534e-04 - val_loss: 5.0713e-05
Epoch 3/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 2.6201e-04 - val_loss: 5.0070e-05
Epoch 4/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 1.5140e-04 - val_loss: 2.8724e-05
Epoch 5/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - loss: 1.6696e-04 - val_loss: 6.4884e-05
Epoch 6/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 1.3114e-04 - val_loss: 4.3708e-05
Epoch 7/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 9.9010e-05 - val_loss: 3.6807e-05
Epoch 8/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - loss: 1.3298e-04 - val_loss: 3.9527e-05
Epoch 9/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - loss: 1.1312e-04 - val_loss: 3.6569e-05
Epoch 10/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 9.7790e-05 - val_loss: 1.2951e-04
Epoch 11/100
847/847 ━━━━━━━━━━━━━━━━━━━━ 9s 11

In [17]:
# Step 14: Make predictions
predictions = model.predict(X_test)

236/236 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step


In [18]:
# Step 15: Inverse transform only the Close price using the trained scaler
y_test_scaled = scaler_close.inverse_transform(y_test.reshape(-1, 1)).flatten()
predictions_scaled = scaler_close.inverse_transform(predictions.reshape(-1, 1)).flatten()

In [19]:
# Step 16: Evaluate the model
mae = mean_absolute_error(predictions_scaled, y_test_scaled)
mape = mean_absolute_percentage_error(predictions_scaled, y_test_scaled)

# Define a percentage threshold for accuracy
threshold_percentage = 5  # 5% tolerance

# Calculate percentage errors
percentage_errors = np.abs((y_test_scaled - predictions_scaled) / y_test_scaled) * 100

# Count predictions within the threshold
acc = np.mean(percentage_errors <= threshold_percentage) * 100

# Alternative accuracy metric
acc2 = (1 - mape) * 100

In [20]:
# Step 17: Print performance metrics
print(f"Mean Absolute Error = {mae}")
print(f"Mean Absolute Percentage Error = {mape*100:.2f}%")
print(f"Accuracy with threshold = {acc:.2f}%")
print(f"Accuracy = {acc2:.2f}%")

Mean Absolute Error = 17.77577018142301
Mean Absolute Percentage Error = 15.03%
Accuracy with threshold = 34.44%
Accuracy = 84.97%
